# Tune the best ML model for PPC

The best = the one with the lowest average MAPE score from "models_filter"

Preprocessing and feature selection is performed after splitting the data (therefore, only the samples in the training data are available for these steps). 

In [1]:
#import libraries
import pandas as pd
import numpy as np
from IPython.display import display

from functools import partial #to pass parameters to function inside another function
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn import svm

In [2]:
#import self-made functions
%run /Users/mariekececilia/Documents/master_thesis_code/methods.ipynb

In [3]:
#set a random seed to make reproducible results (used in mutual information)
seed = 0

## Load and clean data

In [4]:
#load transcriptional and fluxomic data
gexp, flux = load_gerosa()

#clean the transcriptional data (remove duplicates)
gexp, groups = clean_gexp_g(gexp)

#define features
X = gexp

#define targets and extract their fluxes
target_labels = ['R_PPC']
targets = dict()
for target_label in target_labels:
    targets[target_label] = flux[target_label]

## Create models/pipelines

Choose selection parameters:

In [5]:
cov_filters = [0.1] #same as in the best model
k_numbers = [15, 10, 5] #same k as in the best model + 1 larger and 1 smaller k

Choose learning algorithms:

In [6]:
algorithms = dict({
    'lr': linear_model.LinearRegression(),
    'lasso_a1': linear_model.Lasso(alpha = 1), #this makes very bad predictions
    'lasso_a0.1': linear_model.Lasso(alpha = 0.1),
    'lasso_a0.05': linear_model.Lasso(alpha = 0.05), #this one causes warning, does not converge
    'lasso_a0.001': linear_model.Lasso(alpha = 0.001), #this one causes warning, does not converge
    'ridge_a1': linear_model.Ridge(alpha = 1),
    'ridge_a0.1': linear_model.Ridge(alpha = 0.1),
    'ridge_a0.05': linear_model.Ridge(alpha = 0.05),
    'ridge_a0.001': linear_model.Ridge(alpha = 0.001),
    })

Make combinations:

In [7]:
def get_pipelines(algorithm, cov_filters, k_numbers):
    models = dict()

    for cov_filter in cov_filters:  
        
        for k in k_numbers:
            #selection by f – same as in the best model (skipping the other scoring method)
            steps = [
                ('filter', CoVSelector(p = cov_filter)),
                ('selection', SelectKBest(score_func = f_regression,
                                          k = k)),
                ('standardize', StandardScaler()),
                ('m', algorithm)
            ]
            models['v%.0f_f%.0f' % (cov_filter*100, k)] = Pipeline(steps=steps)

    return models

In [8]:
models = dict()

for name, algorithm in algorithms.items():
     models[name]= get_pipelines(algorithm = algorithm, cov_filters = cov_filters, k_numbers = k_numbers)

## Learning

In [9]:
# evaluate the models and store results in nested dicts: target -> learning algorithm -> scores/preds of all selection combinations
scores = dict()
preds = dict()
for y_name, y in targets.items():
    model_scores = dict()
    model_preds = dict()
    for model_name, pipelines in models.items():
        #get scores
        model_scores[model_name] = compare_models(pipelines, X, y)
        #get predictions
        model_preds[model_name] = get_models_preds(pipelines, X, y)
    scores[y_name] = model_scores
    preds[y_name] = model_preds

/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.951e-04, tolerance: 5.907e-04
  model = cd_fast.enet_coordinate_descent(
/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.951e-04, tolerance: 5.907e-04
  model = cd_fast.enet_coordinate_descent(
/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

### Save data to use in other notebooks

Scores and predictions are visualized in the Notebook 'compare_models', which also merges the results with the results from all other models built.

In [10]:
scores_tune_ppc = scores
preds_tune_ppc= preds
%store scores_tune_ppc
%store preds_tune_ppc

Stored 'scores_tune_ppc' (dict)
Stored 'preds_tune_ppc' (dict)


### Scores and predictions

In [11]:
for y in targets.keys():
    for algorithm in algorithms.keys():
        print()
        print(y + '\t(target)') 
        print(algorithm, '\t(learning algorithm)') 
        print('–> summary of all cv split scores for each combination of selection methods:')
        df = scores[y][algorithm][1].sort_values(by = 'average')
        df.index.name = 'Selection combo'
        df.columns.name = 'Score'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all individual cv-split scores for each combination of selection methods:')
        df = scores[y][algorithm][0]
        df.columns.name = 'Selection combo'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all predictions for each combination of selection methods (+ actual values):')
        df = preds[y][algorithm]
        df.index.name = 'Test set'
        df.columns.name = 'Selection combo'
        display(df)


R_PPC	(target)
lr 	(learning algorithm)
–> summary of all cv split scores for each combination of selection methods:


Score,average,std
Selection combo,,
v10_f10,0.165893,0.061271
v10_f15,0.367378,0.500012
v10_f5,0.596608,0.728406



R_PPC	(target)
lr	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,v10_f15,v10_f10,v10_f5
Test set,,,
Acetate,0.306289,0.158488,0.182489
Fructose,0.250583,0.185369,0.890140
Galactose,1.672588,0.115320,2.281964
Glucose,0.149871,0.128289,0.018354
Glycerol,0.068355,0.209116,0.243763
Gluconate,0.146459,0.191467,0.020764
Pyruvate,0.266286,0.062126,0.988631
Succinate,0.078589,0.276967,0.146758



R_PPC	(target)
lr	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


...

Score,average,std
Selection combo,,
v10_f10,0.164848,0.060185
v10_f15,0.366857,0.500260
v10_f5,0.486524,0.596424



R_PPC	(target)
ridge_a0.001	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,v10_f15,v10_f10,v10_f5
Test set,,,
Acetate,0.302916,0.154620,0.176747
Fructose,0.250588,0.185825,0.849593
Galactose,1.672895,0.120797,1.876255
Glucose,0.149761,0.131029,0.010355
Glycerol,0.068018,0.208410,0.185302
Gluconate,0.145534,0.184873,0.020387
Pyruvate,0.266981,0.059273,0.643661
Succinate,0.078166,0.273954,0.129890



R_PPC	(target)
ridge_a0.001	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


Selection combo,actual,v10_f15,v10_f10,v10_f5
Test set,,,,
Acetate,1.774145,2.311563,2.048464,2.087721
Fructose,3.545669,2.657166,2.886794,0.533295
Galactose,0.377386,1.008714,0.422974,1.085459
Glucose,2.453331,2.085918,2.131873,2.478735
Glycerol,1.376469,1.282845,1.089599,1.631530
Gluconate,1.943350,2.226174,1.584078,1.982969
Pyruvate,2.489449,3.154085,2.637005,0.887087
Succinate,2.016110,1.858520,1.463789,2.277983
